In [1]:
from build_Case1 import build_Case1
from build_Case2 import build_Case2
import numpy as np
from scipy.interpolate import interp1d
import pickle
import pandas as pd
import uuid
import time
from pathlib import Path
import io
import os
import json

In [2]:
rootPath = '/nobackup/jakravit/data/'
runPath = rootPath + 'HE60/run/'
modPath = rootPath + 'mod6/'
batch_name = 'case1ModTest'
clearIOPprofiles = True
numRuns = 10

if clearIOPprofiles:
    # clear iop profiles
    %rm -rf /nobackup/jakravit/data/HE60/iop_profiles/{batch_name}
    %mkdir /nobackup/jakravit/data/HE60/iop_profiles/{batch_name}
    # clear batch files
    %rm -rf /nobackup/jakravit/data/HE60/run/batch
    %mkdir /nobackup/jakravit/data/HE60/run/batch
    # clear runlist
    if os.path.exists('/nobackup/jakravit/data/HE60/run/{}_runlist.txt'.format(batch_name)):
        %rm /nobackup/jakravit/data/HE60/run/{batch_name}_runlist.txt  
    Path('/nobackup/jakravit/data/HE60/run/{}_runlist.txt'.format(batch_name)).touch()
    # clear outputs
    %rm -rf /nobackup/jakravit/data/HE60/output/EcoLight/excel
    %mkdir /nobackup/jakravit/data/HE60/output/EcoLight/excel   
    # clear batchruns file
    # %rm /nobackup/jakravit/data/HE60/run/batchruns.sh 
    # with io.open(runPath + 'batchruns.sh', 'a+') as f:
    #     f.write('#!/bin/bash\n') 
    # clear pickles
    %rm -rf /nobackup/jakravit/data/HE60/pickles
    %mkdir /nobackup/jakravit/data/HE60/pickles 

In [3]:
# Data paths 

# phytoplankton SIOP spectral library
phyPath = rootPath+'EAP_phytoplankton_dataset/PFTs_v0817.p'
with open(phyPath, 'rb') as fp:
    phy_library = pickle.load(fp)

# NAP spectral libraries
napPath = rootPath+'EAP_NAP_dataset/NAP_v0817_or.p'
with open(napPath, 'rb') as fp:
    datanap = pickle.load(fp)  

# detpath = rootPath+'EAP_NAP_dataset/det_V1.p'
# with open(detpath, 'rb') as fp:
#     datadet = pickle.load(fp) 

# Benthic library
benthic_lib = pd.read_csv(rootPath+'benthic_spec_library_FT_V0826.csv')

# adjacency library
adj_lib = pd.read_csv(rootPath+'adjacency_spectra_V2.csv')

# aeronet library
aero_lib = pd.read_csv(rootPath+'aeronet_invdata_match.csv')

# modtran json input template
with open('TEMP_20cases.json', 'r') as f:
    modtp = json.load(f)

In [4]:
rows = []
# cols = []
uidList = []
for k in range(numRuns):
    iops, col, row = build_Case1(phy_library, datanap, benthic_lib, adj_lib, aero_lib)
    rows.append(row)
    # cols.append(col)

    #% TOTAL IOPS

    # depth
    depth = iops['Depth']['Depth']
    depth = depth.reshape(-1, 1)
    xfactor = iops['Depth']['xfactor']
    dlen = len(depth)

    # chl profile
    chlProfile = [iops['Phyto']['TotChl'] * x for x in xfactor]
    chlProfile = np.asarray(chlProfile).reshape(-1, 1)
    foot = np.hstack((-1,0))
    chlzProfile = np.hstack((depth,chlProfile))
    chlzProfile = np.vstack((chlzProfile,foot))

    # absorption
    atotphy = iops['Phyto']['a_tot']
    atotmin = iops['Min']['a_tot']
    atotdet = iops['Det']['a_tot']
    atotdom = iops['CDOM']['a_tot'][63:]
    atot = atotphy + atotmin + atotdet + atotdom
    a_tot = [atot * x for x in xfactor]
    a_totphy = [atotphy * x for x in xfactor]
    a_totmin = [atotmin * x for x in xfactor]
    a_totdet = [atotdet * x for x in xfactor]
    a_totdom = [atotdom * x for x in xfactor]

    # Scatter
    btotphy = iops['Phyto']['b_tot']
    btotmin = iops['Min']['b_tot']
    btotdet = iops['Det']['b_tot']
    btot = btotphy + btotmin + btotdet
    # b_tot = [btot * x for x in xfactor]

    # attenuation
    ctotphy = atotphy + btotphy
    ctotmin = atotmin + btotmin
    ctotdet = atotdet + btotdet
    ctotdom = atotdom # no scattering
    ctot = atot + btot
    c_tot = [ctot * x for x in xfactor]
    c_totphy = [ctotphy * x for x in xfactor]
    c_totmin = [ctotmin * x for x in xfactor]
    c_totdet = [ctotdet * x for x in xfactor]
    c_totdom = [ctotdom * x for x in xfactor]

    # backscattering
    bbtotphy = iops['Phyto']['bb_tot']
    bbtotmin = iops['Min']['bb_tot']
    bbtotdet = iops['Det']['bb_tot']
    bbtot = bbtotphy + bbtotmin + bbtotdet
    bb_tot = [bbtot * x for x in xfactor]
    bb_totphy = [bbtotphy * x for x in xfactor]
    bb_totmin = [bbtotmin * x for x in xfactor]
    bb_totdet = [bbtotdet * x for x in xfactor]
    
    #% PREPARE FOR WRITING TO .TXT FILE IN HYDROLIGHT FORMAT

    # lambda
    l = np.arange(400,902.5,2.5)

    llen = len(l)
    # llen2 = str(llen)
    # l2 = [str(x) for x in l]
    line11 = np.hstack((llen,l)).reshape(1,-1)
    # AC data
    foot = np.hstack((-1,[0]*(llen*2)))
    # only phy
    acProfile_phy = np.hstack((depth,a_totphy,c_totphy))
    acProfile_phy = np.vstack((acProfile_phy,foot))
    # only min
    acProfile_min = np.hstack((depth,a_totmin,c_totmin))
    acProfile_min = np.vstack((acProfile_min,foot))
    # only det
    acProfile_det = np.hstack((depth,a_totdet,c_totdet))
    acProfile_det = np.vstack((acProfile_det,foot))
    # only dom
    acProfile_dom = np.hstack((depth,a_totdom,c_totdom))
    acProfile_dom = np.vstack((acProfile_dom,foot))
    # all components
    acProfile = np.hstack((depth,a_tot,c_tot))
    acProfile = np.vstack((acProfile,foot))  

    # bb data
    foot = np.hstack((-1,[0]*(l)))
    # only phy
    bbProfile_phy = np.hstack((depth,bb_totphy))
    bbProfile_phy = np.vstack((bbProfile_phy,foot))
    # only min
    bbProfile_min = np.hstack((depth,bb_totmin))
    bbProfile_min = np.vstack((bbProfile_min,foot))
    # only det
    bbProfile_det = np.hstack((depth,bb_totdet))
    bbProfile_det = np.vstack((bbProfile_det,foot))
    # all components
    bbProfile = np.hstack((depth,bb_tot))
    bbProfile = np.vstack((bbProfile,foot))

    #% WRITE TO TXT
    
    import string
    import random
    import os 

    iopPath = rootPath+'HE60/iop_profiles/{}/'.format(batch_name)
    if not os.path.exists(iopPath):
        os.mkdir(iopPath)
        
    def random_uid():
        alphabet = string.ascii_lowercase + string.digits
        return ''.join(random.choices(alphabet, k=10))
    uid = random_uid()
    
    # create new sunglint and bottom reflectance sim uid
    uidSG = uid + '_SG'
    uidBR = uid + '_BR'
    uidSGBR = uid + '_SG_BR'
    uidList.append(uid)
    uidList.append(uidSG)
    uidList.append(uidBR)
    uidList.append(uidSGBR)

    # sname.append(uid)
    iopzpath = iopPath+'{}/'.format(uid)
    if not os.path.exists(iopzpath):
        os.mkdir(iopzpath)
    # os.mkdir(rootPath + 'HE60/iop_profiles/{}/{}/'.format(batch_name,uid))
    # iopzpath = rootPath + 'HE60/iop_profiles/{}/{}/'.format(batch_name,uid)

    # acData

    # fnames
    fnames = {'total': {'acfname': 'acDatatot_' + uid + '.txt',
                        'bbfname': 'bbDatatot_' + uid + '.txt',
                        'acdata': acProfile,
                        'bbdata': bbProfile},
              'phyto': {'acfname': 'acDataphy_' + uid + '.txt',
                        'bbfname': 'bbDataphy_' + uid + '.txt',
                        'acdata': acProfile_phy,
                        'bbdata': bbProfile_phy},
              'min':   {'acfname': 'acDatamin_' + uid + '.txt',
                        'bbfname': 'bbDatamin_' + uid + '.txt',
                        'acdata': acProfile_min,
                       'bbdata': bbProfile_min},
              'det':   {'acfname': 'acDatadet_' + uid + '.txt',
                        'bbfname': 'bbDatadet_' + uid + '.txt',
                        'acdata': acProfile_det,
                        'bbdata': bbProfile_det},
              }


    # write benthic data
    benthicTot = iops['Benthic']['Tot'].values.reshape(-1,1)
    foot = np.hstack((-1,-1))
    wl = l.reshape(-1,1)
    benthicProfile = np.hstack((wl,benthicTot))
    benthicProfile = np.vstack((benthicProfile,foot))
    header = '\n'.join(['Bottom Reflectance Spectrum',
                        'SWIPE HE data files',
                        '#',
                        '#','#','#','#','#','#','#\n'])    
    with open(os.path.join(iopzpath,'benthicData_{}.txt'.format(uid)),'w') as f:
        f.writelines(header)
        np.savetxt(f,benthicProfile,delimiter='\t')    
    
    # write chl data
    header = '\n'.join(['CHL CONC DATA PROFILE',
                        'SWIPE HE data files',
                        '#',
                        '#','#','#','#','#','#','#\n'])
    with open(os.path.join(iopzpath,'chlzData_{}.txt'.format(uid)),'w') as f:
        f.writelines(header)
        np.savetxt(f,chlzProfile,delimiter='\t')
    
    # write AC data
    fmt1 = '%d,'
    fmt2 = '%1.1f,'*len(l)
    fmt3 = fmt1 +fmt2
    header = '\n'.join(['TOTAL AC DATA PROFILE',
                        'SWIPE HE data files',
                        'Does not include water!',
                        '#','#','#','#','#','#','#\n'])
    for fname in fnames:
        with open(os.path.join(iopzpath,fnames[fname]['acfname']),'w') as f:
            f.writelines(header)
            np.savetxt(f,line11, fmt=fmt3, delimiter='\t') # ,fmt='%d, %1.1f'
            np.savetxt(f,fnames[fname]['acdata'],delimiter='\t')
    
    # write bb data
    header = '\n'.join(['TOTAL BB DATA PROFILE',
                        'SWIPE HE data files',
                        'Does not include water!',
                        '#','#','#','#','#','#','#\n'])
    for fname in fnames:
        with open(os.path.join(iopzpath,fnames[fname]['bbfname']),'w') as f:
            f.writelines(header)
            np.savetxt(f,line11,fmt=fmt3, delimiter='\t') # fmt='%d, %1.1f'
            np.savetxt(f,fnames[fname]['bbdata'],delimiter='\t')

    # write iroot file        
    from write_iroot import iroot
    # iroot without bottom refl
    iroot(l, iops, batch_name, uid, rootPath, br=False)
    # iroot with bottom refl
    iroot(l, iops, batch_name, uid, rootPath, br=True)
    
    # write modtran json files
    from write_modJSON import modjson
    modjson(modtp, iops, aero_lib, batch_name, uid, modPath)
    
    # write pickle file
    with open(rootPath + 'HE60/pickles/{}.p'.format(uid), 'wb') as f:
        pickle.dump(iops,f)
            
sims = pd.DataFrame(np.vstack((rows)), columns=col, index=uidList)
sims.to_csv(rootPath + 'HE60/batch_data/{}_inputs.csv'.format(batch_name))
print ('DONE!')

DONE!


In [5]:
sims

,TotChl,ph_a_tot_400.0,ph_a_tot_402.5,ph_a_tot_405.0,ph_a_tot_407.5,ph_a_tot_410.0,ph_a_tot_412.5,ph_a_tot_415.0,ph_a_tot_417.5,ph_a_tot_420.0,...,aphyCy_887.5,aphyCy_890.0,aphyCy_892.5,aphyCy_895.0,aphyCy_897.5,aphyCy_900.0,Fl685_amp,Br_included,Sg_included,Sg_amplitude
bseb6caw0n,0.585,0.019200158519084284,0.019595163108299653,0.019883049193779163,0.02025458290873831,0.02056168465991626,0.020859923518705262,0.021069635180064883,0.021240378691198936,0.021316916622057473,...,0.0,0.0,0.0,0.0,0.0,0.0,nan,0,0,0
bseb6caw0n_SG,0.585,0.019200158519084284,0.019595163108299653,0.019883049193779163,0.02025458290873831,0.02056168465991626,0.020859923518705262,0.021069635180064883,0.021240378691198936,0.021316916622057473,...,0.0,0.0,0.0,0.0,0.0,0.0,nan,0,1,nan
bseb6caw0n_BR,0.585,0.019200158519084284,0.019595163108299653,0.019883049193779163,0.02025458290873831,0.02056168465991626,0.020859923518705262,0.021069635180064883,0.021240378691198936,0.021316916622057473,...,0.0,0.0,0.0,0.0,0.0,0.0,nan,1,0,0
bseb6caw0n_SG_BR,0.585,0.019200158519084284,0.019595163108299653,0.019883049193779163,0.02025458290873831,0.02056168465991626,0.020859923518705262,0.021069635180064883,0.021240378691198936,0.021316916622057473,...,0.0,0.0,0.0,0.0,0.0,0.0,nan,1,1,nan
3amlrubt5l,1.074,0.02752367045492437,0.027574509508582104,0.027585576759208127,0.02762142504006304,0.02763361630441584,0.027628570301997286,0.02759486044962496,0.02753743158094623,0.02745066958856545,...,8.525537101120386e-08,9.711998967281033e-08,1.2562845547359163e-07,1.5368403065876451e-07,2.007207943045651e-07,2.552238343529285e-07,nan,0,0,0
3amlrubt5l_SG,1.074,0.02752367045492437,0.027574509508582104,0.027585576759208127,0.02762142504006304,0.02763361630441584,0.027628570301997286,0.02759486044962496,0.02753743158094623,0.02745066958856545,...,8.525537101120386e-08,9.711998967281033e-08,1.2562845547359163e-07,1.5368403065876451e-07,2.007207943045651e-07,2.552238343529285e-07,nan,0,1,nan
3amlrubt5l_BR,1.074,0.02752367045492437,0.027574509508582104,0.027585576759208127,0.02762142504006304,0.02763361630441584,0.027628570301997286,0.02759486044962496,0.02753743158094623,0.02745066958856545,...,8.525537101120386e-08,9.711998967281033e-08,1.2562845547359163e-07,1.5368403065876451e-07,2.007207943045651e-07,2.552238343529285e-07,nan,1,0,0
3amlrubt5l_SG_BR,1.074,0.02752367045492437,0.027574509508582104,0.027585576759208127,0.02762142504006304,0.02763361630441584,0.027628570301997286,0.02759486044962496,0.02753743158094623,0.02745066958856545,...,8.525537101120386e-08,9.711998967281033e-08,1.2562845547359163e-07,1.5368403065876451e-07,2.007207943045651e-07,2.552238343529285e-07,nan,1,1,nan
jq689cn67a,0.9,0.022200722858324872,0.02306305102050233,0.02336076720722865,0.02384763509648625,0.02421051388154504,0.02465668406329302,0.02503277818366932,0.02545900568338058,0.02582686888649352,...,5.850697377343105e-07,5.602699012838928e-07,5.858098902465161e-07,5.291598045687234e-07,4.7463158105767895e-07,4.3063143726889374e-07,nan,0,0,0
jq689cn67a_SG,0.9,0.022200722858324872,0.02306305102050233,0.02336076720722865,0.02384763509648625,0.02421051388154504,0.02465668406329302,0.02503277818366932,0.02545900568338058,0.02582686888649352,...,5.850697377343105e-07,5.602699012838928e-07,5.858098902465161e-07,5.291598045687234e-07,4.7463158105767895e-07,4.3063143726889374e-07,nan,0,1,nan
